In [1]:
import os
import tensorflow as tf
import malaya
import json

In [2]:
negatives = ['negative/' + i for i in os.listdir('negative') if '.json' in i]
positives = ['positive/' + i for i in os.listdir('positive') if '.json' in i]
negatives

['negative/2.json',
 'negative/clean-politifact_real.json',
 'negative/clean-gossipcop_real3.json',
 'negative/clean-gossipcop_real2.json',
 'negative/clean-gossipcop_real1.json',
 'negative/1.json']

In [3]:
import re

tokenizer = malaya.preprocessing._SocialTokenizer().tokenize
accept_tokens = ',-.()"\''

def is_number_regex(s):
    if re.match("^\d+?\.\d+?$", s) is None:
        return s.isdigit()
    return True

def detect_money(word):
    if word[:2] == 'rm' and is_number_regex(word[2:]):
        return True
    else:
        return False

def preprocessing(string):
    tokenized = tokenizer(string)
    tokenized = [w.lower() for w in tokenized if len(w) > 1 or w in accept_tokens]
    tokenized = ['<NUM>' if is_number_regex(w) else w for w in tokenized]
    tokenized = ['<MONEY>' if detect_money(w) else w for w in tokenized]
    return tokenized

def clean_label(label):
    string = re.sub('[^A-Za-z\- ]+', ' ', label)
    return re.sub(r'[ ]+', ' ', string.lower()).strip()

In [4]:
X, Y = [], []

for n in negatives:
    with open(n) as fopen:
        x = json.load(fopen)
    processed = [preprocessing(s) for s in x]
    X.extend(processed)
    Y.extend([0] * len(processed))
    
len(X), len(Y)

(26832, 26832)

In [5]:
for p in positives:
    with open(p) as fopen:
        x = json.load(fopen)
    processed = [preprocessing(s) for s in x]
    X.extend(processed)
    Y.extend([1] * len(processed))
    
len(X), len(Y)

(42023, 42023)

In [6]:
def build_dataset(words, n_words):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [7]:
import itertools
import collections

concat = list(itertools.chain(*X))
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 199359
Most common words [(',', 1088529), ('.', 956019), ('yang', 636546), ('dan', 479145), ('"', 468691), ('untuk', 319102)]
Sample data [436, 2073, 5, 926, 7923, 67, 4, 8, 40, 264] ['demokrat', 'sen', '.', 'al', 'franken', 'berkata', ',', '"', 'kita', 'harus']


In [8]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [9]:
import tensorflow as tf
import numpy as np

In [10]:
def embed_seq(x, vocab_sz, embed_dim, name, zero_pad=True): 
    embedding = tf.get_variable(name, [vocab_sz, embed_dim]) 
    if zero_pad:
        embedding = tf.concat([tf.zeros([1, embed_dim]), embedding[1:, :]], 0) 
    x = tf.nn.embedding_lookup(embedding, x)
    return x

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta


def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size, activation):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x, pad], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          dilation_rate = dilation_rate,
                          activation = activation)
    x = x[:, :-pad_sz, :]
    return x

class Model:
    def __init__(self, dict_size, dimension_output, size_layer, num_layers, kernel_size = 7):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        self.training = tf.placeholder(tf.bool, None)
        x = self.X
        
        with tf.variable_scope('embed'):
            x = embed_seq(x, dict_size, size_layer, 'word')
        x += position_encoding(x)
        
        def residual_block(x, size, rate, block):
            with tf.variable_scope('block_%d_%d' % (block, rate), reuse = False):
                pad_sz = (size - 1) * rate
                with tf.variable_scope('block_tanh'):
                    conv_filter = cnn_block(x, rate, pad_sz, x.shape[2], size, tf.nn.tanh)
                with tf.variable_scope('block_sigmoid'):
                    conv_gate = cnn_block(x, rate, pad_sz, x.shape[2], size, tf.nn.sigmoid)
                out = tf.multiply(conv_filter, conv_gate)
                out = tf.layers.dropout(out, 0.2, training=self.training) 
                with tf.variable_scope('block_out'):
                    out = layer_norm(out)
                    out = tf.layers.conv1d(
                        out,
                        size_layer,
                        kernel_size = 1,
                        strides = 1,
                        padding = 'same',
                        activation = tf.nn.tanh,
                    )
                return tf.add(x, out), out
            
        forward = tf.layers.conv1d(x, size_layer, kernel_size = 1, strides = 1, padding = 'SAME')
        zeros = tf.zeros_like(forward)
        for i in range(num_layers):
            for r in [1, 2, 4, 8, 16]:
                forward, s = residual_block(forward, size=kernel_size, rate=r, block=i)
                zeros = tf.add(zeros,s)
        zeros = layer_norm(zeros)
        self.logits = tf.layers.dense(zeros[:,-1], dimension_output)
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
maxlen = 1000
size_layer = 128
num_layers = 2
learning_rate = 1e-4
batch_size = 32

In [12]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(len(dictionary),2,size_layer,num_layers)
sess.run(tf.global_variables_initializer())

In [13]:
def str_idx(corpus, dic, maxlen, UNK = 3):
    X = np.zeros((len(corpus), maxlen))
    for i in range(len(corpus)):
        for no, k in enumerate(corpus[i][:maxlen][::-1]):
            X[i, -1 - no] = dic.get(k, UNK)
    return X

In [14]:
from sklearn.cross_validation import train_test_split

vectors = str_idx(X,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, Y, test_size = 0.2)

In [15]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.training: True
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.training: False
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/1051 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.632588
time taken: 722.3939571380615
epoch: 0, training loss: 0.676048, training acc: 0.619595, valid loss: 0.666471, valid acc: 0.632588



train minibatch loop:   0%|          | 0/1051 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.632588, current acc: 0.638775
time taken: 720.1892328262329
epoch: 1, training loss: 0.657792, training acc: 0.636372, valid loss: 0.643318, valid acc: 0.638775



train minibatch loop:   0%|          | 0/1051 [00:00<?, ?it/s]

...

time taken: 720.0212905406952
epoch: 6, training loss: 0.230076, training acc: 0.911387, valid loss: 0.683921, valid acc: 0.708439



train minibatch loop:   0%|          | 0/1051 [00:00<?, ?it/s]

time taken: 720.1899311542511
epoch: 7, training loss: 0.208620, training acc: 0.920139, valid loss: 0.936518, valid acc: 0.651727



test minibatch loop: 100%|██████████| 263/263 [00:47<00:00,  6.04it/s, accuracy=0.476, cost=1.41] 

time taken: 720.0596556663513
epoch: 8, training loss: 0.172520, training acc: 0.935689, valid loss: 1.036736, valid acc: 0.622753

break epoch:9



In [16]:
pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
y_predict = []
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    logits = sess.run(model.logits,
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.training: False
            },
        )
    logits = np.argmax(logits,1).tolist()
    y_predict.extend(logits)

test minibatch loop: 100%|██████████| 263/263 [00:47<00:00,  6.11it/s]


In [17]:
from sklearn import metrics

print(metrics.classification_report(test_Y, y_predict, target_names = ['-', '+']))

             precision    recall  f1-score   support

          -       0.79      0.54      0.64      5308
          +       0.49      0.76      0.60      3097

avg / total       0.68      0.62      0.63      8405

